<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-description" data-toc-modified-id="Project-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project description</a></span></li><li><span><a href="#Make-feature-file" data-toc-modified-id="Make-feature-file-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Make feature file</a></span></li><li><span><a href="#Train-test" data-toc-modified-id="Train-test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Train-test</a></span></li>

In [247]:
#%matplotlib notebook
import numpy as np
import pandas as pd
import obspy
import matplotlib
import matplotlib.pyplot as plt
import os
import sys
import random
from itertools import combinations
from datetime import datetime, timedelta
import time
import pickle
from itertools import cycle
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score #new
from sklearn.linear_model import LinearRegression #new
from sklearn.model_selection import KFold #new
from sklearn.model_selection import cross_val_score #new
from sklearn.model_selection import train_test_split
from obspy.clients.filesystem.sds import Client
from scipy import signal
import seaborn as sns
import joblib
import itertools
import glob
from ComputeAttributes_CH_V1 import *
from DF_detections import *

In [248]:
# set plotting default parameters
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.figsize'] = (6.4, 4.8) # if wider plot, only change first value.
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Verdana']
#plt.rcParams['ytick.major.pad']= 2

In [249]:
path_to_raw_data = '/data/wsd03/data_manuela/Illgraben/miniseed/'
file_ending = 'beginning_2017'

# Project description

In this notebook we use a manually assembled event catalog containing slope failure events, earthquakes and noise signals recorded on Stations ILL16, ILL17 and ILL18.
We choose 40 s windows that start several seconds before the event-start in the catalog with a specific overlap (for further explanation see Wenner et al., 2021).
The generated/converted catalog is then used to compute features of the signals in the windows after Provost et al., 2017.
We then train a random forest classifier with manual picked events from 2017. The random forest classifier __distinguishes between two classes: slope failures and noise (including earthquakes)__.
Lastly, we apply the classifier on continuous data (2019) to look at seasonal variations in slope failure occurance and the impact of climatic forcing.

The notebook is structured in three main tasks:
1. Compute features from a snuffler file (pyrocko event picker format) -> 2017
2. Train and test a classifier
3. Apply on continuous data -> 2019

Explanations of the specific steps follow below

# Make feature file

Here, we read in a slightly modified snuffler file containing events picked on all Illgraben stations.


In [250]:
# Mate catalog
df = pd.read_csv('../snuffler_files/catalog_{}.txt'.format(file_ending), delimiter=',')
df.head()

,startdate,starttime,enddate,endtime,duration,bla,class,network
0,2017-06-02,10:02:57.8257,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
1,2017-06-02,10:08:39.0653,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
2,2017-06-02,10:13:14.0537,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
3,2017-06-02,10:22:11.1152,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
4,2017-06-02,10:32:15.0563,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ


In [251]:
# Define window length
wdow = 40

# Get times of continuous noise (class0)
# Snuffler output different than for events with starttime and endtime marked
# To sample continuous noise I only marked one timestamp at which nothing was happening
ct = df[df['duration'].isna()].reset_index(drop=True)
# ct = df[df['endtime'] == '0'].reset_index(drop=True)
ct.head()

,startdate,starttime,enddate,endtime,duration,bla,class,network
0,2017-06-02,10:02:57.8257,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
1,2017-06-02,10:08:39.0653,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
2,2017-06-02,10:13:14.0537,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
3,2017-06-02,10:22:11.1152,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ
4,2017-06-02,10:32:15.0563,NaN,NaN,NaN,NaN,0.0,XP.ILL07..EHZ


In [252]:
# Here I filter to only get events from station ILL08 and None (marked at all stations)
# I do this to avoid sampling noise from station ILL08, which we consider to be one of the most valuable stations for our problem
station_list = ['ILL08']
df7 = df[df['duration'].notna()].reset_index(drop=True)
df7 = df7[(df7['network'] == f'XP.ILL08..EHZ') | (df7['network'] == 'None')].reset_index(drop=True)
df7.head()

,startdate,starttime,enddate,endtime,duration,bla,class,network
0,2017-05-24,06:46:04.2439,2017-05-24,06:46:13.4420,9.1981,NaN,4.0,XP.ILL08..EHZ
1,2017-05-25,08:58:35.1350,2017-05-25,08:59:28.3719,53.2369,NaN,4.0,XP.ILL08..EHZ
2,2017-05-25,09:36:18.7083,2017-05-25,09:37:12.4857,53.7774,NaN,4.0,XP.ILL08..EHZ
3,2017-05-29,11:41:29.2703,2017-05-29,11:41:44.2099,14.9396,NaN,4.0,XP.ILL08..EHZ
4,2017-05-29,16:32:08.2913,2017-05-29,16:32:32.8090,24.5177,NaN,4.0,XP.ILL08..EHZ


In [253]:
# Assemble catalog
classes = []
event_idxs = []
slice_idxs = []
times = []
stations = []
for idx, row in df7.iterrows(): # df7 includes only ILL08 and None stations
    sdate = row['startdate']
    stime = row['starttime']
    stt = obspy.UTCDateTime(f"{sdate}T{stime}")
    edate = row['enddate']
    etime = row['endtime']
    edt = obspy.UTCDateTime(f"{edate}T{etime}")
    cla = row['class']
    station = row['network'][3:8] #ILLxx

    # Start random up to 2/3 of window length before marker starts
    tt = stt - np.random.randint(int((2/3)*wdow)) # random numbers up to 2/3*40 =26.66s 
    j = 0
    while tt < edt-(int((1/3)*wdow)): # End random up to 1/3 of window length (13.33s) before marker ends
        classes.append(cla)
        event_idxs.append(idx)
        slice_idxs.append(j)
        times.append(tt)

        if station == 'e': # in None
            stations.append('ILL08')#(random.choice(station_list))
        else:
            stations.append(station)
        tt += int((1/3)*wdow) # overlap of about 2/3 (original code) & 1/3 (PhD Thesis text)
        j+=1

# Also add continuous noise to new catalog
for idxn, row in ct.iterrows():
    idx += 1 # attention: idx is NOT idxn
    sdate = row['startdate']
    stime = row['starttime']
    stt = obspy.UTCDateTime(f"{sdate}T{stime}")
    cla = 0

    classes.append(cla)
    event_idxs.append(idx)
    slice_idxs.append(0)
    times.append(stt + 0.5*wdow)
    stations.append('ILL08')#(random.choice(station_list))

# Get in dataframe format
dic_re = {'event_idx': event_idxs, 'slice_idx': slice_idxs, 'class': classes, 'mean_time': times, 'station': stations}
cat_re = pd.DataFrame(dic_re)
#cat_re[cat_re['class'] == 1]
print(cat_re.head())

# Save catalog
cat_re.to_csv(f"../catalog/catalog_{wdow}_{file_ending}.csv", index=False)
# output each time a bit different due to random element

   event_idx  slice_idx  class                    mean_time station
0          0          0    4.0  2017-05-24T06:45:43.243900Z   ILL08
1          0          1    4.0  2017-05-24T06:45:56.243900Z   ILL08
2          1          0    4.0  2017-05-25T08:58:23.135000Z   ILL08
3          1          1    4.0  2017-05-25T08:58:36.135000Z   ILL08
4          1          2    4.0  2017-05-25T08:58:49.135000Z   ILL08


#### => it is possible to start here and load the modified catalog file

In [254]:
# Read catalog
cat = pd.read_csv(f'../catalog/catalog_{wdow}_{file_ending}.csv')
cat.head()

,event_idx,slice_idx,class,mean_time,station
0,0,0,4.0,2017-05-24T06:45:43.243900Z,ILL08
1,0,1,4.0,2017-05-24T06:45:56.243900Z,ILL08
2,1,0,4.0,2017-05-25T08:58:23.135000Z,ILL08
3,1,1,4.0,2017-05-25T08:58:36.135000Z,ILL08
4,1,2,4.0,2017-05-25T08:58:49.135000Z,ILL08


In [255]:
def get_mseed(time, client=Client(path_to_raw_data), network = 'XP',
              stations=['ILL06','ILL07','ILL08'],channels=['EHZ'], locations=[''], 
              prepick=0, window_length=24*60*60.):

    st = obspy.Stream()
    for station in stations:
        for location in locations:
            for channel in channels:
                try:           
                    new_st = client.get_waveforms(network, station, location, channel,
                                                obspy.UTCDateTime(time)-prepick, 
                                                obspy.UTCDateTime(time)-prepick+window_length)
                    st += new_st
                except:
                    continue
    return(st)

In [256]:
def create_feature_file(df, wdow, filt):
    """
    Creat attribute files from catalog

    :param df: pandas dataframe containing event_idx, slice_idx, class, mean_time and station

    """

    all_char = {}
    
    # Loop over rows in dataframe
    for idx, row in df.iterrows():
        #print(idx)
        tstring = row['mean_time']
        t = obspy.UTCDateTime(f"{tstring}")
        feats = np.array([])
        
        
        
        st = get_mseed(obspy.UTCDateTime('{}-{}'.format(t.year,t.julday)))
        st.detrend('demean')
        st1 = st.copy()
        if filt == 'yes':
            st1.filter('bandpass', freqmin=1, freqmax=10)
            fi = 'yfilt'
        else:
            fi = 'nfilt'
        st1.trim(t - 0.5*wdow, t + 0.5*wdow) # cut stream to 40s (wdow) segments (20s before t and 20s after t)
        print(st1)                           # t = mean_time of event (from df7)
        for tr in st1:
            att = calculate_all_attributes(tr.data,tr.stats.sampling_rate,0) # Compute all atrributes
            feats = np.append(feats,att)
            print(len(feats))
        feats = np.reshape(feats,(1,len(feats)))
        
        
        ev_type = np.array([int(row['event_idx']),int(row['slice_idx']),int(row['class'])])
        type_att = np.append(ev_type, feats)
        type_att = np.reshape(type_att,(1,len(type_att)))
        type_att = pd.DataFrame(type_att)
        # Save to file
        with open(f'/data/wsd03/data_manuela/Illgraben/feature_files/all_{file_ending}.csv', "a+") as f:
            type_att.to_csv(f, header=False,index=False)

In [261]:
def create_feature_file(df, wdow, filt):
    """
    Creat attribute files from catalog

    :param df: pandas dataframe containing event_idx, slice_idx, class, mean_time and station

    """

    all_char = {}
    
    # Loop over rows in dataframe
    for idx, row in df.iterrows():
        #print(idx)
        tstring = row['mean_time']
        t = obspy.UTCDateTime(f"{tstring}")
        feats = np.array([])
                
        st = get_mseed(obspy.UTCDateTime('{}-{}'.format(t.year,t.julday)))
#         if st == 3: # number of stations
        st.detrend('demean')
        st1 = st.copy()
        if filt == 'yes':
            st1.filter('bandpass', freqmin=1, freqmax=10)
        st1.trim(t - 0.5*wdow, t + 0.5*wdow) # cut stream to 40s (wdow) segments (20s before t and 20s after t)
        print(st1)                           # t = mean_time of event (from df7)
        for tr in st1:
            att = calculate_all_attributes(tr.data,tr.stats.sampling_rate,0) # Compute all atrributes
            feats = np.append(feats,att)
            print(len(feats))
        feats = np.reshape(feats,(1,len(feats)))


        ev_type = np.array([int(row['event_idx']),int(row['slice_idx']),int(row['class'])])
        type_att = np.append(ev_type, feats)
        type_att = np.reshape(type_att,(1,len(type_att)))
        type_att = pd.DataFrame(type_att)
        # Save to file
        with open(f'/data/wsd03/data_manuela/Illgraben/feature_files/all_{file_ending}.csv', "a+") as f:
            type_att.to_csv(f, header=False,index=False)
#         else:
# #             print('{} does not contain data for all stations.'.format(t.julday))
#             pass

In [262]:
# Create feature file --> time consuming
create_feature_file(cat, wdow, 'yfilt')

1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-24T06:45:23.240000Z - 2017-05-24T06:46:03.240000Z | 100.0 Hz, 4001 samples
58
1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-24T06:45:36.240000Z - 2017-05-24T06:46:16.240000Z | 100.0 Hz, 4001 samples
58
1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-25T08:58:03.140000Z - 2017-05-25T08:58:43.130000Z | 100.0 Hz, 4000 samples
58
1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-25T08:58:16.140000Z - 2017-05-25T08:58:56.130000Z | 100.0 Hz, 4000 samples
58
1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-25T08:58:29.140000Z - 2017-05-25T08:59:09.130000Z | 100.0 Hz, 4000 samples
58
1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-25T08:58:42.140000Z - 2017-05-25T08:59:22.130000Z | 100.0 Hz, 4000 samples
58
1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-25T08:58:55.140000Z - 2017-05-25T08:59:35.130000Z | 100.0 Hz, 4000 samples
58
1 Trace(s) in Stream:
XP.ILL08..EHZ | 2017-05-25T09:35:39.710000Z - 2017-05-25T09:36:19.710000Z | 100.0 Hz, 4001 samples
58
1 Trace(

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-05T20:37:57.920000Z - 2017-06-05T20:38:37.920000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-05T20:37:57.920000Z - 2017-06-05T20:38:37.920000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-05T20:37:57.920000Z - 2017-06-05T20:38:37.920000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-06T04:06:13.940000Z - 2017-06-06T04:06:53.940000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-06T04:06:13.940000Z - 2017-06-06T04:06:53.940000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-06T04:06:13.940000Z - 2017-06-06T04:06:53.940000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-06T04:06:26.940000Z - 2017-06-06T04:07:06.940000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-06T04:06:26.940000Z - 2017-06-06T04:07:06.940000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-06T04:06:26.940000Z - 2017-06-06T04:07:06.940000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-08T13:47:42.550000Z - 2017-06-08T13:48:22.550000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-08T13:47:42.550000Z - 2017-06-08T13:48:22.550000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-08T13:47:42.550000Z - 2017-06-08T13:48:22.550000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-08T13:47:55.550000Z - 2017-06-08T13:48:35.550000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-08T13:47:55.550000Z - 2017-06-08T13:48:35.550000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-08T13:47:55.550000Z - 2017-06-08T13:48:35.550000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-08T13:48:08.550000Z - 2017-06-08T13:48:48.550000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-08T13:48:08.550000Z - 2017-06-08T13:48:48.550000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-08T13:48:08.550000Z - 2017-06-08T13:48:48.550000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T13:14:11.500000Z - 2017-06-14T13:14:51.500000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T13:14:11.500000Z - 2017-06-14T13:14:51.500000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T13:14:11.500000Z - 2017-06-14T13:14:51.500000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T13:20:06.190000Z - 2017-06-14T13:20:46.190000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T13:20:06.190000Z - 2017-06-14T13:20:46.190000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T13:20:06.190000Z - 2017-06-14T13:20:46.190000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T13:20:19.190000Z - 2017-06-14T13:20:59.190000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T13:20:19.190000Z - 2017-06-14T13:20:59.190000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T13:20:19.190000Z - 2017-06-14T13:20:59.190000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T18:50:49.790000Z - 2017-06-14T18:51:29.790000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T18:50:49.790000Z - 2017-06-14T18:51:29.790000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T18:50:49.790000Z - 2017-06-14T18:51:29.790000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T18:52:09.110000Z - 2017-06-14T18:52:49.110000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T18:52:09.110000Z - 2017-06-14T18:52:49.110000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T18:52:09.110000Z - 2017-06-14T18:52:49.110000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T18:54:02.700000Z - 2017-06-14T18:54:42.700000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T18:54:02.700000Z - 2017-06-14T18:54:42.700000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T18:54:02.700000Z - 2017-06-14T18:54:42.700000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

...

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-12T22:49:33.410000Z - 2017-06-12T22:50:13.410000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-12T22:49:33.410000Z - 2017-06-12T22:50:13.410000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-12T22:49:33.410001Z - 2017-06-12T22:50:13.410001Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-13T00:06:02.900000Z - 2017-06-13T00:06:42.900000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-13T00:06:02.900000Z - 2017-06-13T00:06:42.900000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-13T00:06:02.900000Z - 2017-06-13T00:06:42.900000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-13T01:06:26.180000Z - 2017-06-13T01:07:06.180000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-13T01:06:26.180000Z - 2017-06-13T01:07:06.180000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-13T01:06:26.180000Z - 2017-06-13T01:07:06.180000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T21:51:41.400000Z - 2017-06-14T21:52:21.400000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T21:51:41.400000Z - 2017-06-14T21:52:21.400000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T21:51:41.400000Z - 2017-06-14T21:52:21.400000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T23:12:12.440000Z - 2017-06-14T23:12:52.440000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T23:12:12.440000Z - 2017-06-14T23:12:52.440000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T23:12:12.440000Z - 2017-06-14T23:12:52.440000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-14T23:56:29.510000Z - 2017-06-14T23:57:09.510000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-14T23:56:29.510000Z - 2017-06-14T23:57:09.510000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-14T23:56:29.510000Z - 2017-06-14T23:57:09.510000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-16T04:47:36.890000Z - 2017-06-16T04:48:16.890000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-16T04:47:36.890000Z - 2017-06-16T04:48:16.890000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-16T04:47:36.890000Z - 2017-06-16T04:48:16.890000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-16T05:48:00.170000Z - 2017-06-16T05:48:40.170000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-16T05:48:00.170000Z - 2017-06-16T05:48:40.170000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-16T05:48:00.170000Z - 2017-06-16T05:48:40.170000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-16T06:32:17.240000Z - 2017-06-16T06:32:57.240000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-16T06:32:17.240000Z - 2017-06-16T06:32:57.240000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-16T06:32:17.240000Z - 2017-06-16T06:32:57.240000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-17T14:56:46.880000Z - 2017-06-17T14:57:26.880000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-17T14:56:46.880000Z - 2017-06-17T14:57:26.880000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-17T14:56:46.880000Z - 2017-06-17T14:57:26.880000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-17T16:01:11.710000Z - 2017-06-17T16:01:51.710000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-17T16:01:11.710000Z - 2017-06-17T16:01:51.710000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-17T16:01:11.710000Z - 2017-06-17T16:01:51.710000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-17T17:05:36.540000Z - 2017-06-17T17:06:16.540000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-17T17:05:36.540000Z - 2017-06-17T17:06:16.540000Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-17T17:05:36.540000Z - 2017-06-17T17:06:16.540000Z | 100.0 Hz, 4001 samples
58
116
174
3 Trace(s)

3 Trace(s) in Stream:
XP.ILL06..EHZ | 2017-06-18T22:43:39.090000Z - 2017-06-18T22:44:19.090000Z | 100.0 Hz, 4001 samples
XP.ILL07..EHZ | 2017-06-18T22:43:39.089989Z - 2017-06-18T22:44:19.089989Z | 100.0 Hz, 4001 samples
XP.ILL08..EHZ | 2017-06-18T22:43:39.090000Z - 2017-06-18T22:44:19.090000Z | 100.0 Hz, 4001 samples
58
116
174


In [259]:
# Read feature file
df = pd.read_csv(f'/data/wsd03/data_manuela/Illgraben/feature_files/all_{file_ending}.csv', names=range(177)) # 58*N_sta+3
print(file_ending)
print(np.shape(df))

beginning_2017
(591, 177)


In [260]:
df

,0,1,2,3,4,5,6,7,8,9,...,167,168,169,170,171,172,173,174,175,176
0,0.0,0.0,4.0,40.01,20.569099,45.413096,265.733333,43.558720,28.134153,0.342472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,4.0,40.01,12.507184,31.865396,2.042586,21.645753,15.873836,0.250515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,0.0,4.0,40.00,5.286637,5.848421,49.632911,9.172503,19.521486,0.146934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,4.0,40.00,7.427659,9.961383,8.029345,8.137521,11.242831,0.077474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,2.0,4.0,40.00,6.660431,8.490721,1.294894,6.656690,9.432106,0.113646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,312.0,0.0,0.0,40.01,1.369153,1.366931,5.735690,2.835271,3.255510,0.011129,...,7.0,28.0,1.714286,0.428571,18.0,23.0,1.277778,16.171783,13.783741,29.955524
587,313.0,0.0,0.0,40.01,2.256354,2.243760,0.123245,3.116112,2.684791,0.098661,...,9.0,23.0,2.666667,1.043478,64.0,10.0,0.156250,16.656041,14.615006,31.271047
588,314.0,0.0,0.0,40.01,2.406959,2.421994,1.948416,2.986419,2.672809,0.018724,...,1.0,2.0,2.000000,1.000000,26.0,9.0,0.346154,15.497021,14.310750,29.807772
589,315.0,0.0,0.0,40.01,1.831801,1.845357,2.135580,3.340541,3.151526,0.145779,...,6.0,28.0,4.833333,1.035714,20.0,4.0,0.200000,18.049816,22.381892,40.431708


In [201]:
# Read feature file
df = pd.read_csv(f'/data/wsd03/data_manuela/Illgraben/feature_files/all_{file_ending}.csv', names=range(177)) # 58*N_sta+3
print(file_ending)
print(np.shape(df))

beginning_2017
(591, 177)


In [202]:
df

,0,1,2,3,4,5,6,7,8,9,...,167,168,169,170,171,172,173,174,175,176
0,0.0,0.0,4.0,40.01,20.569099,45.413096,265.733333,43.558720,28.134153,0.342472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,4.0,40.01,12.507184,31.865396,2.042586,21.645753,15.873836,0.250515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,0.0,4.0,40.00,5.286637,5.848421,49.632911,9.172503,19.521486,0.146934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,4.0,40.00,7.427659,9.961383,8.029345,8.137521,11.242831,0.077474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,2.0,4.0,40.00,6.660431,8.490721,1.294894,6.656690,9.432106,0.113646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,312.0,0.0,0.0,40.01,1.369153,1.366931,5.735690,2.835271,3.255510,0.011129,...,7.0,28.0,1.714286,0.428571,18.0,23.0,1.277778,16.171783,13.783741,29.955524
587,313.0,0.0,0.0,40.01,2.256354,2.243760,0.123245,3.116112,2.684791,0.098661,...,9.0,23.0,2.666667,1.043478,64.0,10.0,0.156250,16.656041,14.615006,31.271047
588,314.0,0.0,0.0,40.01,2.406959,2.421994,1.948416,2.986419,2.672809,0.018724,...,1.0,2.0,2.000000,1.000000,26.0,9.0,0.346154,15.497021,14.310750,29.807772
589,315.0,0.0,0.0,40.01,1.831801,1.845357,2.135580,3.340541,3.151526,0.145779,...,6.0,28.0,4.833333,1.035714,20.0,4.0,0.200000,18.049816,22.381892,40.431708


# Train-test

In [203]:
# List of attributes we want to use for classification
attribute_names = ['duration', 'RappMaxMean','RappMaxMedian', 'AsDec', 'KurtoSig', \
                    'KurtoEnv', 'SkewnessSig', 'SkewnessEnv', 'CorPeakNumber', 'INT1', \
                    'INT2', 'INT_RATIO', 'ES[0]', 'ES[1]', 'ES[2]', 'ES[3]', 'ES[4]', 'KurtoF[0]', \
                    'KurtoF[1]', 'KurtoF[2]', 'KurtoF[3]', 'KurtoF[4]', 'DistDecAmpEnv', \
                    'env_max/duration(Data,sps)', 'MeanFFT', 'MaxFFT', 'FmaxFFT', \
                    'FCentroid', 'Fquart1', 'Fquart3', 'MedianFFT', 'VarFFT', 'NpeakFFT', \
                    'MeanPeaksFFT', 'E1FFT', 'E2FFT', 'E3FFT', 'E4FFT', 'gamma1', 'gamma2', \
                    'gammas', 'SpecKurtoMaxEnv', 'SpecKurtoMedianEnv', 'RATIOENVSPECMAXMEAN', \
                    'RATIOENVSPECMAXMEDIAN', 'DISTMAXMEAN', 'DISTMAXMEDIAN', 'NBRPEAKMAX', \
                    'NBRPEAKMEAN', 'NBRPEAKMEDIAN', 'RATIONBRPEAKMAXMEAN', \
                    'RATIONBRPEAKMAXMED', 'NBRPEAKFREQCENTER', 'NBRPEAKFREQMAX', \
                    'RATIONBRFREQPEAKS', 'DISTQ2Q1', 'DISTQ3Q2', 'DISTQ3Q1']

## Unsplited labeled Data (v5)

In [204]:
def train_test_grouped(df1, features, size):
    """
    Split into training and test data set such that events recorded on multiple stations will either be in train or in test data set
    Additional to sklearn function add feature values to X-files instead of only the event-index
    
    :param df1: whole data set
    :param gr: grouped per event index
    :param features: list of feautre names
    """
    # Get event idx and targets
    gr = df1.groupby('event_idx').first() # df inclueds only first time window per event
    idxs = np.asarray(gr.index) # index 0-290 (len = 291)
    y = np.asarray(gr['event_class']) # class 0 or 1 (len = 291)
    # Split training and validation data
    X_train, X_test, y_train, y_test = train_test_split(idxs, y, test_size=size,random_state=42)
    df_tr = df1.loc[df1['event_idx'].isin(X_train)] # df with feature values TRAIN
    df_va = df1.loc[df1['event_idx'].isin(X_test)] # df with feature values TEST
    X_train = np.asarray(df_tr[features]) # feature values to TRAIN
    y_train = np.asarray(df_tr['event_class']) # classes to TRAIN (only 0 and 1)
    X_test = np.asarray(df_va[features]) # feature values to TEST
    y_test = np.asarray(df_va['event_class']) # classes to TEST (only 0 and 1)
    return X_train, X_test, y_train, y_test

In [245]:
gr = catalog.groupby('event_idx').first()
gr

,slice_idx,event_class,duration,RappMaxMean,RappMaxMedian,AsDec,KurtoSig,KurtoEnv,SkewnessSig,SkewnessEnv,...,NBRPEAKMEAN,NBRPEAKMEDIAN,RATIONBRPEAKMAXMEAN,RATIONBRPEAKMAXMED,NBRPEAKFREQCENTER,NBRPEAKFREQMAX,RATIONBRFREQPEAKS,DISTQ2Q1,DISTQ3Q2,DISTQ3Q1
event_idx,,,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,40.01,20.569099,45.413096,265.733333,43.558720,28.134153,0.342472,4.792911,...,0.0,0.0,0.000000,0.000000,15.0,24.0,1.600000,18.446573,15.723525,34.170098
1.0,0.0,0.0,40.00,5.286637,5.848421,49.632911,9.172503,19.521486,0.146934,3.529462,...,0.0,3.0,0.000000,0.000000,44.0,41.0,0.931818,21.559377,17.233302,38.792679
2.0,0.0,0.0,40.01,5.930528,8.205586,4.564673,7.712090,7.672325,0.247795,1.895298,...,2.0,12.0,1.000000,0.166667,23.0,7.0,0.304348,10.535684,13.995995,24.531679
3.0,0.0,0.0,40.01,11.093845,19.554634,3.933416,18.160485,15.505463,0.095811,3.171612,...,2.0,6.0,2.500000,0.833333,5.0,5.0,1.000000,12.894226,18.491574,31.385799
4.0,0.0,0.0,40.01,5.489416,6.190620,0.000000,7.426104,10.237366,0.063556,2.213162,...,0.0,1.0,0.000000,0.000000,26.0,9.0,0.346154,17.789311,17.784811,35.574123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312.0,0.0,0.0,40.01,1.369153,1.366931,5.735690,2.835271,3.255510,0.011129,0.039825,...,20.0,47.0,0.350000,0.148936,35.0,18.0,0.514286,13.334233,32.109062,45.443295
313.0,0.0,0.0,40.01,2.256354,2.243760,0.123245,3.116112,2.684791,0.098661,0.012489,...,15.0,29.0,0.933333,0.482759,37.0,0.0,0.000000,18.118567,29.193511,47.312078
314.0,0.0,0.0,40.01,2.406959,2.421994,1.948416,2.986419,2.672809,0.018724,0.039154,...,27.0,51.0,0.407407,0.215686,43.0,7.0,0.162791,21.020368,29.456765,50.477133


In [246]:
gr['event_class'].value_counts()[1.0] # SF 60, noise 1627

1

In [241]:
df_final

,event_idx,slice_idx,event_class,duration,RappMaxMean,RappMaxMedian,AsDec,KurtoSig,KurtoEnv,SkewnessSig,...,NBRPEAKMEAN,NBRPEAKMEDIAN,RATIONBRPEAKMAXMEAN,RATIONBRPEAKMAXMED,NBRPEAKFREQCENTER,NBRPEAKFREQMAX,RATIONBRFREQPEAKS,DISTQ2Q1,DISTQ3Q2,DISTQ3Q1
0,0.0,0.0,4.0,40.01,20.569099,45.413096,265.733333,43.558720,28.134153,0.342472,...,0.0,0.0,0.000000,0.000000,15.0,24.0,1.600000,18.446573,15.723525,34.170098
1,0.0,1.0,4.0,40.01,12.507184,31.865396,2.042586,21.645753,15.873836,0.250515,...,1.0,3.0,3.000000,1.000000,18.0,33.0,1.833333,19.264337,12.734723,31.999060
2,1.0,0.0,4.0,40.00,5.286637,5.848421,49.632911,9.172503,19.521486,0.146934,...,0.0,3.0,0.000000,0.000000,44.0,41.0,0.931818,21.559377,17.233302,38.792679
3,1.0,1.0,4.0,40.00,7.427659,9.961383,8.029345,8.137521,11.242831,0.077474,...,3.0,10.0,1.000000,0.300000,14.0,26.0,1.857143,16.990547,15.100264,32.090812
4,1.0,2.0,4.0,40.00,6.660431,8.490721,1.294894,6.656690,9.432106,0.113646,...,4.0,15.0,0.750000,0.200000,1.0,9.0,9.000000,10.513434,13.846492,24.359926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682,312.0,0.0,0.0,40.01,3.441553,3.578573,0.415281,4.025727,5.867156,0.013107,...,7.0,28.0,1.714286,0.428571,18.0,23.0,1.277778,16.171783,13.783741,29.955524
1683,313.0,0.0,0.0,40.01,3.069524,3.169933,0.026161,3.481105,4.972838,0.005792,...,9.0,23.0,2.666667,1.043478,64.0,10.0,0.156250,16.656041,14.615006,31.271047
1684,314.0,0.0,0.0,40.01,3.931884,4.178080,0.025897,5.434915,7.876373,0.007880,...,1.0,2.0,2.000000,1.000000,26.0,9.0,0.346154,15.497021,14.310750,29.807772
1685,315.0,0.0,0.0,40.01,3.406601,3.425518,1.075207,4.112643,4.252655,0.038014,...,6.0,28.0,4.833333,1.035714,20.0,4.0,0.200000,18.049816,22.381892,40.431708


In [230]:
catalog

,event_idx,slice_idx,event_class,duration,RappMaxMean,RappMaxMedian,AsDec,KurtoSig,KurtoEnv,SkewnessSig,...,NBRPEAKMEAN,NBRPEAKMEDIAN,RATIONBRPEAKMAXMEAN,RATIONBRPEAKMAXMED,NBRPEAKFREQCENTER,NBRPEAKFREQMAX,RATIONBRFREQPEAKS,DISTQ2Q1,DISTQ3Q2,DISTQ3Q1
0,0.0,0.0,0.0,40.01,20.569099,45.413096,265.733333,43.558720,28.134153,0.342472,...,0.0,0.0,0.000000,0.000000,15.0,24.0,1.600000,18.446573,15.723525,34.170098
1,0.0,1.0,0.0,40.01,12.507184,31.865396,2.042586,21.645753,15.873836,0.250515,...,1.0,3.0,3.000000,1.000000,18.0,33.0,1.833333,19.264337,12.734723,31.999060
2,1.0,0.0,0.0,40.00,5.286637,5.848421,49.632911,9.172503,19.521486,0.146934,...,0.0,3.0,0.000000,0.000000,44.0,41.0,0.931818,21.559377,17.233302,38.792679
3,1.0,1.0,0.0,40.00,7.427659,9.961383,8.029345,8.137521,11.242831,0.077474,...,3.0,10.0,1.000000,0.300000,14.0,26.0,1.857143,16.990547,15.100264,32.090812
4,1.0,2.0,0.0,40.00,6.660431,8.490721,1.294894,6.656690,9.432106,0.113646,...,4.0,15.0,0.750000,0.200000,1.0,9.0,9.000000,10.513434,13.846492,24.359926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682,312.0,0.0,0.0,40.01,3.441553,3.578573,0.415281,4.025727,5.867156,0.013107,...,7.0,28.0,1.714286,0.428571,18.0,23.0,1.277778,16.171783,13.783741,29.955524
1683,313.0,0.0,0.0,40.01,3.069524,3.169933,0.026161,3.481105,4.972838,0.005792,...,9.0,23.0,2.666667,1.043478,64.0,10.0,0.156250,16.656041,14.615006,31.271047
1684,314.0,0.0,0.0,40.01,3.931884,4.178080,0.025897,5.434915,7.876373,0.007880,...,1.0,2.0,2.000000,1.000000,26.0,9.0,0.346154,15.497021,14.310750,29.807772
1685,315.0,0.0,0.0,40.01,3.406601,3.425518,1.075207,4.112643,4.252655,0.038014,...,6.0,28.0,4.833333,1.035714,20.0,4.0,0.200000,18.049816,22.381892,40.431708


In [205]:
# Assemble whole header of feautre file
header = []
h = ['event_idx', 'slice_idx', 'event_class']
header = h + attribute_names + attribute_names + attribute_names # 3 times because 3 stations
print(len(header))

177


In [217]:
# Load feature file and set new header
# df_ff = pd.read_csv('/data/wsd03/data_manuela/Illgraben/feature_files/all_40s_hSNR_yfilt_v4_original.csv'.format(file_ending), header = None, names=range(177))
df_ff = pd.read_csv('/data/wsd03/data_manuela/Illgraben/feature_files/all_{}.csv'.format(file_ending), header = None, names=range(177))

df_ff.columns = header
print(np.shape(df_ff)) # the three stations are behind each other (in x direction)

# Now we have to list the different stations below each other (in y direction)
df_final = pd.DataFrame()
for n in [3, 61, 119]:
    df_new = df_ff.iloc[:,:3].join(df_ff.iloc[:,n:n+58]) # first three cols copy and than add features
    df_final = df_final.append(df_new)
print(np.shape(df_final))

df_f = df_final[df_final['duration'].notna()].reset_index(drop=True) # keep only stations with data for the time window 
df_final = df_f

(591, 177)
(1773, 61)


/tmp/ipykernel_3457877/503501315.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_new)
/tmp/ipykernel_3457877/503501315.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_new)
/tmp/ipykernel_3457877/503501315.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_new)


In [218]:
np.unique(df_final.event_class)

array([0., 1., 3., 4.])

In [212]:
# RFC can only handle two classes -> merge classes
def combine_classes_catalog_2(cat):
    cat.loc[cat['event_class'] != 3 , 'event_class'] = 0 # Earthqauke, Thunder, ambient Noise --> Noise
    cat.loc[cat['event_class'] == 3 , 'event_class'] = 1 # Slope failures from 3 to 1
    return cat

catalog = combine_classes_catalog_2(df_final)
catalog

,event_idx,slice_idx,event_class,duration,RappMaxMean,RappMaxMedian,AsDec,KurtoSig,KurtoEnv,SkewnessSig,...,NBRPEAKMEAN,NBRPEAKMEDIAN,RATIONBRPEAKMAXMEAN,RATIONBRPEAKMAXMED,NBRPEAKFREQCENTER,NBRPEAKFREQMAX,RATIONBRFREQPEAKS,DISTQ2Q1,DISTQ3Q2,DISTQ3Q1
0,0.0,0.0,0.0,40.01,20.569099,45.413096,265.733333,43.558720,28.134153,0.342472,...,0.0,0.0,0.000000,0.000000,15.0,24.0,1.600000,18.446573,15.723525,34.170098
1,0.0,1.0,0.0,40.01,12.507184,31.865396,2.042586,21.645753,15.873836,0.250515,...,1.0,3.0,3.000000,1.000000,18.0,33.0,1.833333,19.264337,12.734723,31.999060
2,1.0,0.0,0.0,40.00,5.286637,5.848421,49.632911,9.172503,19.521486,0.146934,...,0.0,3.0,0.000000,0.000000,44.0,41.0,0.931818,21.559377,17.233302,38.792679
3,1.0,1.0,0.0,40.00,7.427659,9.961383,8.029345,8.137521,11.242831,0.077474,...,3.0,10.0,1.000000,0.300000,14.0,26.0,1.857143,16.990547,15.100264,32.090812
4,1.0,2.0,0.0,40.00,6.660431,8.490721,1.294894,6.656690,9.432106,0.113646,...,4.0,15.0,0.750000,0.200000,1.0,9.0,9.000000,10.513434,13.846492,24.359926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682,312.0,0.0,0.0,40.01,3.441553,3.578573,0.415281,4.025727,5.867156,0.013107,...,7.0,28.0,1.714286,0.428571,18.0,23.0,1.277778,16.171783,13.783741,29.955524
1683,313.0,0.0,0.0,40.01,3.069524,3.169933,0.026161,3.481105,4.972838,0.005792,...,9.0,23.0,2.666667,1.043478,64.0,10.0,0.156250,16.656041,14.615006,31.271047
1684,314.0,0.0,0.0,40.01,3.931884,4.178080,0.025897,5.434915,7.876373,0.007880,...,1.0,2.0,2.000000,1.000000,26.0,9.0,0.346154,15.497021,14.310750,29.807772
1685,315.0,0.0,0.0,40.01,3.406601,3.425518,1.075207,4.112643,4.252655,0.038014,...,6.0,28.0,4.833333,1.035714,20.0,4.0,0.200000,18.049816,22.381892,40.431708


In [219]:
np.unique(catalog.event_class)

array([0., 1.])

In [220]:
X_train, X_test, y_train, y_test = train_test_grouped(catalog, attribute_names, size=0.3) # 30% data to test

print(np.shape(X_train)) # feature file
print(np.shape(y_train)) # classification (only 0 and 1)
print(np.shape(X_test)) # feature file
print(np.shape(y_test)) # classification (only 0 and 1)

print(len(catalog['event_class']==1))
print(np.where(catalog['event_class']==1))
print(np.shape(np.where(y_test==1)))
print(np.shape(np.where(y_train==1)))

(1059, 58)
(1059,)
(628, 58)
(628,)
1687
(array([ 117,  118,  119,  120,  121,  122,  123,  124,  125,  126,  127,
        128,  129,  130,  131,  132,  133,  134,  135,  136,  685,  686,
        687,  688,  689,  690,  691,  692,  693,  694,  695,  696,  697,
        698,  699,  700,  701,  702,  703,  704, 1233, 1234, 1235, 1236,
       1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247,
       1248, 1249, 1250, 1251, 1252]),)
(1, 60)
(1, 0)


In [214]:
np.where(np.isnan(X_train))

(array([], dtype=int64), array([], dtype=int64))

In [227]:
np.unique(y_train)

array([0.])

## Splited labeled data (v4)

In [127]:
# Read train and test feautre file
plot_ending = 'test_v4_bigsf_clf2'
#catalog = pd.read_csv(path_to_raw_data[:-9] + 'feature_files/train_features_yfilt_40s_hSNR_v4_with_thunder.csv')
catalog = pd.read_csv(path_to_raw_data[:-9] + 'feature_files/train_features_yfilt_40s_hSNR_v4_with_thunder_without_bigsf.csv')
catalog_test = pd.read_csv(path_to_raw_data[:-9] + 'feature_files/test_features_yfilt_40s_hSNR_v4_with_thunder.csv')

def combine_classes_catalog_2(cat):
    cat.loc[cat['event_class'] == 2, 'event_class'] = 0 # Earthqauke Illgraben to Noise
    return cat

catalog = combine_classes_catalog_2(catalog)
catalog_test = combine_classes_catalog_2(catalog_test)
print(np.shape(catalog))
catalog.head()

(620, 61)


,event_idx,slice_idx,event_class,duration,RappMaxMean,RappMaxMedian,AsDec,KurtoSig,KurtoEnv,SkewnessSig,...,NBRPEAKMEAN,NBRPEAKMEDIAN,RATIONBRPEAKMAXMEAN,RATIONBRPEAKMAXMED,NBRPEAKFREQCENTER,NBRPEAKFREQMAX,RATIONBRFREQPEAKS,DISTQ2Q1,DISTQ3Q2,DISTQ3Q1
0,0,0,0,40.01,11.621308,13.382875,0.000000,29.192959,58.224163,0.654186,...,0,0,0.000000,0.000000,43,23,0.534884,5.813852,5.007588,10.821439
1,0,1,0,40.01,12.989940,41.775964,4.243775,24.769869,18.235417,0.617135,...,3,9,1.333333,0.444444,43,16,0.372093,4.841585,4.302575,9.144160
2,2,0,0,40.01,8.805683,13.153355,9.232737,12.392325,12.534898,0.263876,...,0,2,0.000000,0.000000,45,10,0.222222,4.941336,5.119590,10.060926
3,2,1,0,40.01,5.517401,8.359824,1.366056,5.482842,4.470063,0.100531,...,5,22,1.600000,0.363636,51,9,0.176471,4.576330,4.914586,9.490916
4,2,2,0,40.01,4.076638,4.514342,92.046512,3.746714,3.910707,0.047172,...,7,53,1.142857,0.150943,45,27,0.600000,4.473828,4.226824,8.700652


In [128]:
gr_train = catalog.groupby('event_idx').first() # takes only first timewindow from event
gr_test = catalog_test.groupby('event_idx').first() # takes only first timewindow from event
gr_test[gr_test['event_class'] == 1]

,slice_idx,event_class,duration,RappMaxMean,RappMaxMedian,AsDec,KurtoSig,KurtoEnv,SkewnessSig,SkewnessEnv,...,NBRPEAKMEAN,NBRPEAKMEDIAN,RATIONBRPEAKMAXMEAN,RATIONBRPEAKMAXMED,NBRPEAKFREQCENTER,NBRPEAKFREQMAX,RATIONBRFREQPEAKS,DISTQ2Q1,DISTQ3Q2,DISTQ3Q1
event_idx,,,,,,,,,,,,,,,,,,,,,
17,0,1,40.01,3.596649,3.894292,0.035455,3.132553,3.944306,0.007406,0.831002,...,3,14,2.000000,0.428571,21,33,1.571429,4.245574,4.105572,8.351146
20,0,1,40.01,9.632125,29.351504,4.857980,12.222148,9.015058,0.200590,2.326823,...,1,9,3.000000,0.333333,23,9,0.391304,4.415077,4.968837,9.383914
24,0,1,40.01,7.936242,39.820574,4.549237,11.392583,7.632915,0.085739,2.092004,...,3,8,1.000000,0.375000,40,22,0.550000,3.878818,3.616063,7.494881
25,0,1,40.01,6.030819,7.589484,17.872642,6.582485,9.183023,0.034780,2.072771,...,0,2,0.000000,0.000000,25,22,0.880000,4.819584,3.897068,8.716653
26,0,1,40.01,8.148813,34.018740,10.497126,10.511834,7.006029,0.144457,1.963308,...,2,8,1.500000,0.375000,35,33,0.942857,3.442060,3.130805,6.572865
27,0,1,40.01,3.352804,3.605667,1.795947,3.221219,3.831803,0.040959,0.853840,...,9,48,1.111111,0.208333,33,31,0.939394,6.196108,4.830335,11.026443
29,0,1,40.01,9.378785,25.950413,0.000000,13.723142,9.579943,0.507181,2.516687,...,0,0,0.000000,0.000000,49,16,0.326531,5.281842,4.585080,9.866923
52,0,1,40.01,7.347712,9.820201,1.357690,9.156820,11.795604,0.326415,2.592355,...,2,15,1.000000,0.133333,16,7,0.437500,4.370576,4.763583,9.134160
61,0,1,40.01,5.355649,7.201601,0.684632,6.299165,6.442957,0.074038,1.752272,...,1,12,5.000000,0.416667,28,17,0.607143,4.482328,4.785084,9.267412


In [129]:
# take all attributes
X_train = np.asarray(catalog[attribute_names])
y_train = np.asarray(catalog['event_class'])
X_test = np.asarray(catalog_test[attribute_names])
y_test = np.asarray(catalog_test['event_class'])

print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

#print(len(catalog['event_class']==1))
#print(np.where(catalog['event_class']==1))
#print(np.shape(np.where(y_test==1)))
#print(np.shape(np.where(y_train==1)))

(620, 58)
(620,)
(263, 58)
(263,)


In [8]:
#X_train= df[df.columns.difference(['Year','X'])] # drop specific columns in df

## RF-model

In [215]:
# Train balanced random forest classifier
clf = BalancedRandomForestClassifier(n_estimators=1200,criterion='gini',sampling_strategy='majority', max_features='sqrt', \
                                     n_jobs=-1, min_samples_leaf = 1, max_depth=10, min_samples_split=20, \
                                     oob_score=False, bootstrap=True, class_weight=None,random_state=10)

In [216]:
# Fit model
clf.fit(X_train, y_train)
# Predict test data set
y_pred = clf.predict(X_test)
# Get probabilities
probas = clf.predict_proba(X_test)  
# Print confision matrix
print(confusion_matrix(y_test, y_pred))

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
# save model
filename = '../model/RF_{}.model'.format(plot_ending)
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
# load model
filename = '../model/RF_v4_bigsf_clf2.model'
clf = pickle.load(open(filename, 'rb'))
clf

In [51]:
## take features with importance 0.005 or bigger
## get feature importance
#importances = clf.feature_importances_
#importances.sort()
#importances
##print(sum(clf.feature_importances_))
#df_imp = pd.DataFrame({'imp': clf.feature_importances_, 'label': attribute_names}).sort_values(by='imp', ascending=False)
#
## drop features with an importance smaller than 0.005
#indexNames = df_imp[df_imp['imp'] < 0.0025].index
#df_imp.drop(indexNames , inplace=True)
#imp_list = df_imp['label'].to_list()
#imp_list
#
#X_train = np.asarray(catalog[imp_list])
#y_train = np.asarray(catalog['event_class'])
#X_test = np.asarray(catalog_test[imp_list])
#y_test = np.asarray(catalog_test['event_class'])
#
#print(np.shape(X_train))
#print(np.shape(y_train))
#print(np.shape(X_test))
#print(np.shape(y_test))
#
## Fit model
#clf.fit(X_train, y_train)
## Predict test data set
#y_pred = clf.predict(X_test)
## Get probabilities
#probas = clf.predict_proba(X_test)  
## Print confision matrix
#print(confusion_matrix(y_test, y_pred))

(620, 56)
(620,)
(263, 56)
(263,)
